# PyHealth GAMENet Reproduction Study

## Imports

First, import python libraries, pyhealth, and related pyhealth libraries

In [1]:
import argparse
import sys
import pandas
import json

import pyhealth

from pyhealth.datasets import MIMIC4Dataset, MIMIC3Dataset
from pyhealth.tasks import drug_recommendation_mimic4_fn, drug_recommendation_mimic3_fn
# import dataloader related functions
from pyhealth.datasets.splitter import split_by_patient
from pyhealth.datasets import split_by_patient, get_dataloader
# import gamenet model
from pyhealth.models import RETAIN, GAMENet
# import trainer
from pyhealth.trainer import Trainer


/home/ram/code/school/dl4h/final/submit/GAMENet/pyhealth/.v2/lib/python3.8/site-packages/pyhealth/trainer.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


Next, import our custom libraries we've designed for this study

In [2]:
from drug_rec_task import drug_recommendation_mimic4_no_hist, drug_recommendation_mimic4_no_proc
from model import ModelWrapper
from alt_gamenets import GAMENetNoHist, GAMENetNoProc
from mimic import MIMIC4, MIMICWrapper

# import our constants
from constants import (
    DEV,
    EPOCHS, LR, DECAY_WEIGHT,
    DRUG_REC_TN, ALL_TASKS,
    GN_KEY, RT_KEY,
    MODEL_TYPES_PER_TASK, RETAIN_FEATS_PER_TASK,
    GAMENET_EXP, RETAIN_EXP,
    SCORE_KEY, DPV_KEY, DDI_RATE_KEY,
    BASE_DDI_RATE
)

## Load Data

We use the `MIMIC4` data class from the `mimic` import.
Another option would be to import the `MIMIC3` data class and use that as the `dataset` below.
For this purpose, we just use `MIMIC4` and use that to load the data and prepare it with the appropriate tasks.

The data class (either `MIMIC4` or `MIMIC3` decides where the data root is.
By default, it reads in data from `./hiddendata/extracted/{mimic3/4}/`.
So, for MIMIC4 data the data would need to be in: `./hiddendata/extracted/mimic4/`.
This can be changed by either modifying the MIMIC4 class directly, or modifying the data root default in the `constants` file.

In [3]:
# whether to read in "dev" mode or not
DEV = False

In [4]:
# save data in the ./hiddendata/extracted/ directory
## this uses MIMIC4, so place data in ./hiddendata/extracted/mimic4/
## could use MIMIC3 with from mimic import MIMIC3 and placing data in ./hiddendata/extracted/mimic3/
dataset = MIMIC4
mimic = MIMICWrapper(datasource=dataset, tasks=dataset.all_tasks())
mimic_data = mimic.load_data(dev=DEV)
drug_task_data = mimic.drug_task_data()
dataloaders = mimic.create_dataloaders()

reading mimic4 data...
---DATA STATS FOR mimic4 DATA---
stat

Statistics of base dataset (dev=False):
	- Dataset: MIMIC4Dataset
	- Number of patients: 180733
	- Number of visits: 431231
	- Number of visits per patient: 2.3860
	- Number of events per visit in diagnoses_icd: 11.0296
	- Number of events per visit in procedures_icd: 1.5518
	- Number of events per visit in prescriptions: 54.2354

info

dataset.patients: patient_id -> <Patient>

<Patient>
    - visits: visit_id -> <Visit> 
    - other patient-level info
    
    <Visit>
        - event_list_dict: table_name -> List[Event]
        - other visit-level info
    
        <Event>
            - code: str
            - other event-level info

***run task: drug_recommendation


Generating samples for drug_recommendation_mimic4_fn: 100%|███████████| 180733/180733 [00:36<00:00, 4890.30it/s]


{'visit_id': '22595853', 'patient_id': '10000032', 'conditions': [['5723', '78959', '5715', '07070', '496', '29680', '30981', 'V1582']], 'procedures': [['5491']], 'drugs': ['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B'], 'drugs_all': [['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B']]}
***run task: no_hist


Generating samples for drug_recommendation_mimic4_no_hist: 100%|██████| 180733/180733 [00:39<00:00, 4550.12it/s]


{'visit_id': '22595853', 'patient_id': '10000032', 'conditions': [['5723', '78959', '5715', '07070', '496', '29680', '30981', 'V1582']], 'procedures': [['5491']], 'drugs': ['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B'], 'drugs_all': [['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B']]}
***run task: no_proc


Generating samples for drug_recommendation_mimic4_no_proc: 100%|██████| 180733/180733 [00:45<00:00, 3955.30it/s]


{'visit_id': '22595853', 'patient_id': '10000032', 'conditions': [['5723', '78959', '5715', '07070', '496', '29680', '30981', 'V1582']], 'drugs': ['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B'], 'drugs_all': [['B01A', 'J07B', 'A12B', 'C03D', 'C03C', 'N02B', 'J05A', 'R03A', 'N07B', 'R03B']]}


In [5]:
drug_task_data

{'drug_recommendation': <pyhealth.datasets.sample_dataset.SampleDataset at 0x7f969fa65f10>,
 'no_hist': <pyhealth.datasets.sample_dataset.SampleDataset at 0x7f96698b7e80>,
 'no_proc': <pyhealth.datasets.sample_dataset.SampleDataset at 0x7f966875ce50>}

## Create DDI Matrices

In order to calculate DDI Rate, we need to create the DDI matrices.
GAMENet models have this built-in, but RETAIN does not.
So, we craft our DDI matrices ahead of time to help us calculate the rate later.

In [6]:
ddi_mats = {}

for taskname in mimic.get_task_names():
    model_type = MODEL_TYPES_PER_TASK[taskname][GN_KEY]
    ddi_mats[taskname] = model_type(drug_task_data[taskname]).generate_ddi_adj()

## Train the Models

In [7]:
retain = {}
gamenet = {}

In [8]:
# baseline
print("---RETAIN TRAINING---")
for taskname,dataloader in dataloaders.items():
    print("--training retain on {} data--".format(taskname))
    # create and train retain model
    retain[taskname] = ModelWrapper(
        drug_task_data[taskname],
        model=MODEL_TYPES_PER_TASK[taskname][RT_KEY],
        feature_keys=RETAIN_FEATS_PER_TASK[taskname],
        experiment="{}_task_{}".format(RETAIN_EXP, taskname)
    )
    retain[taskname].train_model(
        dataloader["train"], dataloader["val"],
        decay_weight=DECAY_WEIGHT,
        learning_rate=LR,
        epochs=EPOCHS
    )

# gamenet
print("---GAMENET TRAINING---")
for taskname,dataloader in dataloaders.items():
    print("--training gamenet on {} data--".format(taskname))
    # create and train gamenet model
    gamenet[taskname] = ModelWrapper(
        drug_task_data[taskname],
        model=MODEL_TYPES_PER_TASK[taskname][GN_KEY],
        experiment="{}_task_{}".format(GAMENET_EXP, taskname)
    )
    gamenet[taskname].train_model(
        dataloader["train"], dataloader["val"],
        decay_weight = DECAY_WEIGHT,
        learning_rate = LR,
        epochs=EPOCHS
    )

---RETAIN TRAINING---
--training retain on drug_recommendation data--
making retain model


RETAIN(
  (embeddings): ModuleDict(
    (conditions): Embedding(19186, 128, padding_idx=0)
    (procedures): Embedding(10605, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (retain): ModuleDict(
    (conditions): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
    (procedures): RETAINLayer(
      (dropout_layer): Dropout(p=0.5, inplace=False)
      (alpha_gru): GRU(128, 128, batch_first=True)
      (beta_gru): GRU(128, 128, batch_first=True)
      (alpha_li): Linear(in_features=128, out_features=1, bias=True)
      (beta_li): Linear(in_features=128, out_features=128, bias=True)
    )
  )
  (fc): Linear(in_features=256, out_features=200, bias=True)
)
Metrics: ['jaccard_samples', 'accuracy', 'hamming_loss', 'precision

Epoch 0 / 20:   0%|          | 0/1842 [00:00<?, ?it/s]

--- Train epoch-0, step-1842 ---
loss: 0.2492
Evaluation: 100%|█████████████████████████████████████████████████████████████| 233/233 [00:04<00:00, 56.13it/s]
/home/ram/code/school/dl4h/final/submit/GAMENet/pyhealth/.v2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
--- Eval epoch-0, step-1842 ---
jaccard_samples: 0.4020
accuracy: 0.0017
hamming_loss: 0.0865
precision_samples: 0.7345
recall_samples: 0.4885
pr_auc_samples: 0.7072
f1_samples: 0.5540
loss: 0.2183
New best accuracy score (0.0017) at epoch-0, step-1842



Epoch 1 / 20:   0%|          | 0/1842 [00:00<?, ?it/s]

...

--- Train epoch-17, step-61866 ---
loss: 0.1647
Evaluation: 100%|█████████████████████████████████████████████████████████████| 435/435 [00:07<00:00, 61.61it/s]
--- Eval epoch-17, step-61866 ---
jaccard_samples: 0.4173
accuracy: 0.0011
hamming_loss: 0.0774
precision_samples: 0.7345
recall_samples: 0.5137
pr_auc_samples: 0.7184
f1_samples: 0.5691
loss: 0.1912



Epoch 18 / 20:   0%|          | 0/3437 [00:00<?, ?it/s]

--- Train epoch-18, step-65303 ---
loss: 0.1634
Evaluation: 100%|█████████████████████████████████████████████████████████████| 435/435 [00:06<00:00, 62.66it/s]
/home/ram/code/school/dl4h/final/submit/GAMENet/pyhealth/.v2/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
--- Eval epoch-18, step-65303 ---
jaccard_samples: 0.4116
accuracy: 0.0008
hamming_loss: 0.0768
precision_samples: 0.7472
recall_samples: 0.4981
pr_auc_samples: 0.7196
f1_samples: 0.5634
loss: 0.1918



Epoch 19 / 20:   0%|          | 0/3437 [00:00<?, ?it/s]

--- Train epoch-19, step-68740 ---
loss: 0.1623
Evaluation: 100%|█████████████████████████████████████████████████████████████| 435/435 [00:07<00:00, 61.64it/s]
--- Eval epoch-19, step-68740 ---
jaccard_samples: 0.4150
accuracy: 0.0011
hamming_loss: 0.0779
precision_samples: 0.7374
recall_samples: 0.5089
pr_auc_samples: 0.7167
f1_samples: 0.5664
loss: 0.1935
Loaded best model


## Evaluate the Models

In [9]:
baseline_result = {}
gamenet_result = {}

In [10]:
# baseline
print("---RETAIN EVALUATION---")
for taskname in mimic.get_task_names():
    print("--eval retain on {} data--".format(taskname))
    test_loader = dataloaders[taskname]["test"]
    baseline_result[taskname] = {}
    baseline_result[taskname][SCORE_KEY] = retain[taskname].evaluate_model(test_loader)
    baseline_result[taskname][SCORE_KEY][DPV_KEY] = retain[taskname].calc_avg_drugs_per_visit(test_loader)
    baseline_result[taskname][SCORE_KEY][DDI_RATE_KEY] = retain[taskname].calc_ddi_rate(
        test_loader, ddi_mats[taskname]
    )
    
# gamenet
print("---GAMENET EVALUATION---")
for taskname in mimic.get_task_names():
    print("--eval gamenet on {} data--".format(taskname))
    test_loader = dataloaders[taskname]["test"]
    gamenet_result[taskname] = {}
    gamenet_result[taskname][SCORE_KEY] = gamenet[taskname].evaluate_model(test_loader)
    gamenet_result[taskname][SCORE_KEY][DPV_KEY] = gamenet[taskname].calc_avg_drugs_per_visit(test_loader)
    gamenet_result[taskname][SCORE_KEY][DDI_RATE_KEY] = gamenet[taskname].calc_ddi_rate(
        test_loader, ddi_mats[taskname]
    )

---RETAIN EVALUATION---
--eval retain on drug_recommendation data--


Evaluation: 100%|█████████████████████████████████████████████████████████████| 230/230 [00:04<00:00, 52.35it/s]


{'jaccard_samples': 0.4540459546771232, 'accuracy': 0.0020420665713702267, 'hamming_loss': 0.07847287454904363, 'precision_samples': 0.7578135655729245, 'recall_samples': 0.5464850884204991, 'pr_auc_samples': 0.7534060235322836, 'f1_samples': 0.6047574833671272, 'loss': 0.18982936526122302}


Evaluation: 100%|█████████████████████████████████████████████████████████████| 230/230 [00:03<00:00, 57.66it/s]


--eval retain on no_hist data--


...

Evaluation: 100%|████████████████████████████████████████████████████████████| 233/233 [00:01<00:00, 230.37it/s]


--eval gamenet on no_proc data--


Evaluation: 100%|█████████████████████████████████████████████████████████████| 432/432 [00:06<00:00, 62.53it/s]


{'jaccard_samples': 0.4232919544592121, 'accuracy': 0.0013046787228644947, 'hamming_loss': 0.07339358727689425, 'precision_samples': 0.7338074555704448, 'recall_samples': 0.5193401782681243, 'pr_auc_samples': 0.7212806490382262, 'f1_samples': 0.5761553823846068, 'loss': 0.1825957858797025}


Evaluation: 100%|█████████████████████████████████████████████████████████████| 432/432 [00:06<00:00, 62.44it/s]


## Display Scores

In [11]:
#results = {RT_KEY: baseline_result, GN_KEY: gamenet_result}
#rtoutfile = open("./results_gamenet.json", 'w')
#gnoutfile = open("./results_retain.json", 'w')
#gnoutfile.write(json.dumps(gamenet_result))
#rtoutfile.write(json.dumps(baseline_result))
#gnoutfile.close()
#rtoutfile.close()

In [12]:
#gn_pd_object = pandas.read_json('./results_gamenet.json', typ='series')
#gn_df = pandas.DataFrame.from_records(gn_pd_object)
#display(gn_df)

,scores,avg_dpv,ddi_rate
0,"{'jaccard_samples': 0.127961854468781, 'accura...",70.103448,0.043540
1,"{'jaccard_samples': 0.133885752813687, 'accura...",70.791667,0.045327
2,"{'jaccard_samples': 0.11004927634283, 'accurac...",69.697368,0.037088


In [16]:
metrics_columns = [
    "accuracy", "precision_samples", "recall_samples",
    "pr_auc_samples", "f1_samples",
    "avg_dpv", "ddi_rate"
]


In [17]:
retain_res_df = pandas.DataFrame.from_dict({(t,v): baseline_result[t][v] 
                            for t in baseline_result.keys()
                            for v in baseline_result[t].keys()},
                           orient='index')
retain_metrics = retain_res_df[metrics_columns]


#task_ids = []
#frames =[]
#
#for taskname, res in gamenet_result.items():
#    print(taskname)
#    print(res)
#    task_ids.append(taskname)
#    frames.append(pandas.DataFrame.from_dict(res, orient="index"))
#    
#pandas.concat(frames, keys=task_ids)

In [18]:
gamenet_res_df = pandas.DataFrame.from_dict({(t,v): gamenet_result[t][v] 
                            for t in gamenet_result.keys()
                            for v in gamenet_result[t].keys()},
                           orient='index')
gamenet_metrics = gamenet_res_df[metrics_columns]

In [19]:
display(retain_metrics)

,,accuracy,precision_samples,recall_samples,pr_auc_samples,f1_samples,avg_dpv,ddi_rate
drug_recommendation,scores,0.002042,0.757814,0.546485,0.753406,0.604757,16.496835,0.062308
no_hist,scores,0.002217,0.775676,0.513036,0.747408,0.590202,14.570555,0.066517
no_proc,scores,0.000834,0.759829,0.464477,0.712385,0.547581,11.440510,0.085610


In [20]:
display(gamenet_metrics)

,,accuracy,precision_samples,recall_samples,pr_auc_samples,f1_samples,avg_dpv,ddi_rate
drug_recommendation,scores,0.002246,0.720763,0.567033,0.737870,0.603262,18.253829,0.060669
no_hist,scores,0.002217,0.754340,0.553618,0.744159,0.603478,17.461967,0.060690
no_proc,scores,0.001305,0.733807,0.519340,0.721281,0.576155,13.977458,0.072855


In [21]:
retain_metrics.to_latex()

'\\begin{tabular}{llrrrrrrr}\n\\toprule\n &  & accuracy & precision_samples & recall_samples & pr_auc_samples & f1_samples & avg_dpv & ddi_rate \\\\\n\\midrule\ndrug_recommendation & scores & 0.002042 & 0.757814 & 0.546485 & 0.753406 & 0.604757 & 16.496835 & 0.062308 \\\\\n\\cline{1-9}\nno_hist & scores & 0.002217 & 0.775676 & 0.513036 & 0.747408 & 0.590202 & 14.570555 & 0.066517 \\\\\n\\cline{1-9}\nno_proc & scores & 0.000834 & 0.759829 & 0.464477 & 0.712385 & 0.547581 & 11.440510 & 0.085610 \\\\\n\\cline{1-9}\n\\bottomrule\n\\end{tabular}\n'

In [22]:
gamenet_metrics.to_latex()

'\\begin{tabular}{llrrrrrrr}\n\\toprule\n &  & accuracy & precision_samples & recall_samples & pr_auc_samples & f1_samples & avg_dpv & ddi_rate \\\\\n\\midrule\ndrug_recommendation & scores & 0.002246 & 0.720763 & 0.567033 & 0.737870 & 0.603262 & 18.253829 & 0.060669 \\\\\n\\cline{1-9}\nno_hist & scores & 0.002217 & 0.754340 & 0.553618 & 0.744159 & 0.603478 & 17.461967 & 0.060690 \\\\\n\\cline{1-9}\nno_proc & scores & 0.001305 & 0.733807 & 0.519340 & 0.721281 & 0.576155 & 13.977458 & 0.072855 \\\\\n\\cline{1-9}\n\\bottomrule\n\\end{tabular}\n'